In [ ]:
from google.colab import drive
mount = '/content/gdrive'
drive.mount(mount)
drive_root = mount + "/My Drive/DPSN-modified/DPSN-modified/SFA_Python-master/test"
%cd $drive_root

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/DPSN-modified/DPSN-modified/SFA_Python-master/test


In [ ]:
!pip install pyts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pwd

/content/gdrive/My Drive/DPSN-modified/DPSN-modified/SFA_Python-master/test


In [ ]:
import sys
import os
import sklearn
import random
from sklearn import preprocessing
sys.path.append(os.getcwd()[:-5])
# from src.timeseries.TimeSeriesLoader import uv_load
# from  src.classification.BOSSEnsembleClassifier import *
import pickle
import pdb
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
from pyts.transformation import BOSS
from os.path import dirname
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from pyts.multivariate.transformation import WEASELMUSE




def save_to_log(sentence,dataset_name):
    father_path = './BOSS_feature_Data_pyts/' + dataset_name
    if not os.path.exists(father_path):
        os.makedirs(father_path)
    path = father_path+ '/' +dataset_name+ '_hyper_parameter_log.txt'
    with open(path, "a") as myfile:
        myfile.write(sentence+'\n')


def WM_feature_data_writer(dataset_name, train_ratio, ind ,X_train_feature, y_train, X_test_feature, y_test, test_accuracy):
    father_path = './WM_feature_Data_pyts/' + dataset_name+ '/'+str(train_ratio)+'/'+str(ind)+'/'
    if not os.path.exists(father_path):
        os.makedirs(father_path)
        

    dictionary = {'X_train_feature': X_train_feature,
                  'y_train': y_train,
                  'X_test_feature': X_test_feature,
                  'y_test': y_test,
                  'Boss_accuracy': test_accuracy}
    
   
    save_path = father_path+ dataset_name + '.npy'
    np.save(save_path, dictionary)



def BOSS_feature_data_loader(dataset_name):
    father_path = dirname(os.getcwd())+ '/test/BOSS_feature_Data/' + dataset_name
    save_path = father_path + '/' + dataset_name + '.npy'
    dictionary = np.load(save_path)
    X_train_feature= dictionary.item().get('X_train_feature')
    y_train = dictionary.item().get('y_train')
    X_test_feature = dictionary.item().get('X_test_feature')
    y_test = dictionary.item().get('y_test')
    test_accuracy = dictionary.item().get('Boss_accuracy')
    return X_train_feature, y_train, X_test_feature, y_test, test_accuracy



def TSC_data_loader(dataset_name):
    Train_dataset = np.loadtxt(os.getcwd()[:-5]+'/datasets/UCRArchive_2018/' + dataset_name + '/' + dataset_name + '_TRAIN.tsv')
    Test_dataset = np.loadtxt(os.getcwd()[:-5]+'/datasets/UCRArchive_2018/' + dataset_name + '/' + dataset_name + '_TEST.tsv')
    Train_dataset = Train_dataset.astype(np.float32)
    Test_dataset = Test_dataset.astype(np.float32)
    

    X_train = Train_dataset[:, 1:]
    y_train = Train_dataset[:, 0:1]

    X_test = Test_dataset[:, 1:]
    y_test = Test_dataset[:, 0:1]
    le = preprocessing.LabelEncoder()
    le.fit(np.squeeze(y_train, axis=1))
    y_train = le.transform(np.squeeze(y_train, axis=1))
    y_test = le.transform(np.squeeze(y_test, axis=1))
    
    #X_train = np.nan_to_num(X_train) 
    #X_test  = np.nan_to_num(X_test)

    return X_train, y_train, X_test, y_test



def BOSSdistance_predict(X_test_feature, X_train_feature,y_train):
    A_predict = np.zeros(X_test_feature.shape[0])
    X_train_featur_matrix = X_train_feature.todense()
    for i in range(X_test_feature.shape[0]):
        ith_X_test_feature = X_test_feature[i,:]
        ith_X_test_feature_matrix=ith_X_test_feature.todense()
        mask = np.where(ith_X_test_feature_matrix == 0, 0, 1)
        dis=np.sum(np.square(np.multiply(mask,(ith_X_test_feature_matrix-X_train_feature))), axis = -1)
        A_predict[i]=y_train[np.argmin(dis)]
    return A_predict



def BOSS_load_Superparameter(dataset_name):
    father_path = dirname(os.getcwd())+ '/test/BOSS_model/'+ dataset_name
    save_path = father_path + '/' + dataset_name + '_model.pkl'
    with open(save_path, 'rb') as input:
        models = pickle.load(input)
    return models


def BOSS_feature_data_writer_sfa(dataset_name, X_train_feature, y_train, X_test_feature, y_test, test_accuracy):
    father_path = './BOSS_feature_Data/' + dataset_name
    if not os.path.exists(father_path):
        os.makedirs(father_path)
        

    dictionary = {'X_train_feature': X_train_feature,
                  'y_train': y_train,
                  'X_test_feature': X_test_feature,
                  'y_test': y_test,
                  'Boss_accuracy': test_accuracy}
    save_path = father_path + '/' + dataset_name + '.npy'
    np.save(save_path, dictionary)

def get_super_parameter(dataset_name):
    
    models = BOSS_load_Superparameter(dataset_name)
    best_model = models[0]
    best_score = models[0][1]
    for i in range(len(models)):
        if models[i][1]>best_score:
            best_model = models[i]
    n_conf = best_model[2]
    windows = best_model[4]
    norm_mean = best_model[3].normMean    
    
    return n_conf, windows, norm_mean



# def get_list():
#     name_list = os.listdir(dirname(os.getcwd())+'/test/BOSS_feature_Data/')
#     name_list.remove('Running_log')
#     name_list.sort()
#     name_list = name_list[0:-1]
#     return name_list


# name_list  = get_list()
name_list = [
 'ECG200'
]


train_ratio_list = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]



In [ ]:
!pwd

/content/gdrive/My Drive/DPSN-modified/DPSN-modified/SFA_Python-master/test


In [ ]:
with open("W_GPA_pool_2018.pkl", 'rb') as f:
    data = pickle.load(f)
data['training_set']['data'].isnull().values.any()

False

In [ ]:
# generate features and labels for train set

temp = data
# temp['gpa']['label']
res = temp['training_set']['data']  # features for training set
res = res.loc[:, ~res.columns.str.endswith(('_post', '_pre', '_demo'))] # remove constant timestamp features

# res.iloc[0, 5:] += 0.0001 # inorder to remove constant timestamp
# res.fillna(1.5, inplace=True) # in order to remove Nan
# res[res['week']==13]
# temp['training_set']['label'] # label for training set
# res[res['PID']==1][res['week']==13].columns[200:205]
res1 = temp['training_set']['label']
res1
comb = pd.concat([res, res1.iloc[:, 1]], axis=1)
comb = comb[comb['week']==13]
# comb.fillna(method='backfill', axis=0)
# code to transform data for features set
PIDs = set(data['training_set']['label']['PID'])
X_train = [comb[comb['PID']==single_pid].iloc[:, 6:-1].T for single_pid in PIDs]
X_train[0:2]
# label
y_train = [comb[comb['PID']==single_pid].iloc[0, -1] for single_pid in PIDs]


# X_train[2].iloc[5, :] += 3.33
# X_train[100].iloc[5, :] += 3.33

X_train = np.array(X_train)
# X_train[:, :, 0] += 1.56
# X_train[:, :, 4] += 2.3
# X_train[:, 16, 2] = 1
X_train.shape

(131, 714, 7)

In [ ]:
array = np.array([[1, 2, 3, 5, 5],
                  [2, 3, 4, 4, 5],
                  [1, 2, 3, 3, 5],
                  [1, 2, 3, 4, 6]])
unique_elements, counts = np.unique(array, return_counts=True, axis=0)
counts
unique_elements

array([[1, 2, 3, 3, 5],
       [1, 2, 3, 4, 6],
       [1, 2, 3, 5, 5],
       [2, 3, 4, 4, 5]])

In [ ]:
# find the columns that need to be removed
num_col = X_train.shape[1]
col_to_remove = []
for i in range(num_col):
  if np.any(np.max(X_train[:, i, :], axis=0) - np.min(X_train[:, i, :], axis=0) <= np.full(7, 40)):
    col_to_remove.append(i)

  # check if more than or equal to 10 are constant
  # for j in range(7):
  #   unique_elements, counts = np.unique(X_train[:, i, j], return_counts=True)
  #   if np.max(counts) >= 50:
  #     col_to_remove.append(i)
  #     break

len(col_to_remove)

# remove those columns
not_col_to_remove = list(set(range(num_col)) - set(col_to_remove))
X_train = X_train[:, not_col_to_remove, :]
X_train.shape
# np.max(X_train[:, 0, :], axis=0)
# X_train[:, [1, 2], :].shape

(131, 222, 7)

In [ ]:
np.max(X_train[:, 1, :], axis=0) - np.min(X_train[:, 1, :], axis=0) <= 40

array([False, False, False, False, False, False, False])

In [ ]:
y_train

[1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0]

In [ ]:
X_train

array([[[  9.        ,  15.        ,  13.        , ...,   0.        ,
          28.239906  ,  28.239906  ],
        [ 37.        ,  16.        ,  24.        , ...,  49.60854074,
          49.60854074,  49.60854074],
        [ 28.        ,  54.        ,  25.        , ...,  45.28224438,
          45.28224438,  45.28224438],
        ...,
        [ 16.09051667, 159.6782    ,  40.43981667, ...,  98.4441768 ,
          98.4441768 ,  98.4441768 ],
        [  2.        ,   8.        ,   2.        , ...,   2.        ,
          18.53662182,  18.53662182],
        [115.        , 115.        , 115.        , ..., 115.        ,
         140.10042975, 140.10042975]],

       [[ 12.        ,  56.        ,  45.        , ...,  78.        ,
          22.        ,  28.239906  ],
        [ 69.        ,  48.        ,  31.        , ...,  92.        ,
          26.        ,   9.        ],
        [ 48.        ,  77.        ,  59.        , ...,  52.        ,
          45.28224438,  77.        ],
        ...,


In [ ]:
np.isnan(X_train).any()

False

In [ ]:
# generate test set data
res = temp['test_set']['data']  # features for test set
# try:
#   res.iloc[0] += 0.0001
# except:
#   pass

res = res.loc[:, ~res.columns.str.endswith(('_post', '_pre', '_demo'))] # remove constant timestamp features

# res[res['week']==13]
# temp['training_set']['label'] # label for training set
# res[res['PID']==1][res['week']==13].columns[200:205]
res1 = temp['test_set']['label']
res1

comb = pd.concat([res, res1.iloc[:, 1]], axis=1)
# comb = comb[comb['week']==13].fillna(method='backfill', axis=1)


# code to transform data for features set
PIDs = set(data['test_set']['label']['PID'])
X_test = [comb[comb['PID']==single_pid].iloc[:, 6:-1].T for single_pid in PIDs]
X_test = np.array(X_test)
X_test.shape
# label
y_test = [comb[comb['PID']==single_pid].iloc[0, -1] for single_pid in PIDs]
X_test = X_test[:, not_col_to_remove, :]
X_test.shape

(57, 222, 70)

In [ ]:

y_test


[1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1]

In [ ]:
data = X_train
# data[:, 5, :] = data[:, 0, :]
data.shape
data[:, 98, 0]

array([ 67.9075,  67.9075,  67.9075,  67.9075,  67.9075,  67.9075,
        67.9075,  67.9075,  67.9075,  67.9075,  67.9075,  67.9075,
        67.9075,  67.9075,  67.9075,  67.9075,  67.9075,  67.9075,
        67.9075,  67.9075,  67.9075,  67.9075,  67.9075,  67.9075,
        67.9075,  67.9075,  67.9075,  67.9075,  67.9075,  67.9075,
        67.9075,  67.9075,  67.9075,  67.9075,  67.9075,  67.9075,
        67.9075,  67.9075,  67.9075,  67.9075,  67.9075,  67.9075,
        67.9075,  67.9075,  67.9075,  67.9075,  67.9075,  67.9075,
        67.9075,  67.9075,  67.9075,  67.9075,  67.9075, 268.125 ,
        67.9075,  67.9075,  67.9075,  67.9075,  67.9075,  67.9075,
        67.9075,  67.9075,  67.9075,  67.9075,  67.9075,  67.9075,
        67.9075,  67.9075,  67.9075,  67.9075,  67.9075,  67.9075,
        67.9075,  67.9075,  67.9075,  67.9075,  67.9075,  67.9075,
        67.9075,  67.9075,  67.9075,  67.9075,  67.9075,  67.9075,
        67.9075,  67.9075,  67.9075,  67.9075,  67.9075,  67.9

In [ ]:
data_label = y_train
data_label

[1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0]

In [ ]:
data = X_train
data_label = np.array(y_train)
data_test = X_test
data_test_label = np.array(y_test)

In [ ]:
# data_test = np.array(X_test)
# data_test.shape

In [ ]:
# data_test_label = np.array(y_test)
# data_test_label

In [ ]:
# # testing
# # Set the random seed for reproducibility
# np.random.seed(0)

# # Number of samples, timestamps, and features
# num_samples = 70
# num_timestamps = 10
# num_features = 6

# # Generate random multivariate time series data
# data = np.random.randn(num_samples, num_features, num_timestamps)
# # data[:, 3] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# array_length = 70  # Specify the desired length of the array

# # Generate a random array of 1s and 2s
# array = [random.choice([1, 2]) for _ in range(array_length)]
# data_label = np.array(array)

# data_test = np.random.randn(10, num_timestamps, num_features)
# data_test_label = np.array([1, 2, 2, 1, 1, 2, 1, 1, 1, 2])
# data.shape

In [ ]:
data.shape

(131, 222, 7)

In [ ]:
data_label.shape

(131,)

In [ ]:
data_test.shape

(57, 222, 70)

In [ ]:
data_test_label.shape

(57,)

In [ ]:
1/np.sqrt(7) * np.sum(np.array([67.9075]*7) * np.exp(-1j * 2 * np.pi * 1 * np.arange(7)))

(179.6663571561187+1.3201669752342353e-13j)

In [ ]:
1/np.sqrt(7) * np.sum(np.array([67.9075]*7) * np.exp(-1j * 2 * np.pi * 1 * np.arange(7)))

(179.6663571561187+1.3201669752342353e-13j)

In [ ]:
# data = np.array([[67] * 7] * 13)
# data

In [ ]:
# data.shape[0] * 7

In [ ]:
# # for i in range(data.shape[1]):
# unique_elements, counts = np.unique(data, return_counts=True)
# # data.shape[0] * 7
# int(counts)
#           # if counts == data.shape[0] * 7:
#           #   skip = True

In [ ]:
test = [[1, 2, 3], [4, 4, 1]]
unique_elements, counts = np.unique(test, return_counts=True)
np.max(counts)

2

In [ ]:
data.shape

(131, 222, 7)

In [ ]:
import copy
X_train = copy.deepcopy(data)
y_train = copy.deepcopy(data_label)
    
# for name in  name_list:
    # n_conf, windows, norm_mean = get_super_parameter(name)
    # X_train, y_train, X_test, y_test = TSC_data_loader(name)
WM_model = WEASELMUSE(strategy='normal', drop_sum=True, window_sizes=[6], word_size=2, n_bins=2)
# pdb.set_trace()

WM_model.fit(data, data_label)
X_train_feature = WM_model.transform(data)

X_test_feature = WM_model.transform(data_test)
# y_predict = BOSSdistance_predict(X_test_feature, X_train_feature, y_train)
# test_accuracy = sklearn.metrics.accuracy_score(np.squeeze(y_test), y_predict)
        
        
# print(name, 'n_conf = ' ,n_conf, 'windows = ', windows, 'norm_mean = ',norm_mean)
# print('pyts accuracy = ', test_accuracy, ' pyts_shape= = ',X_train_feature.shape)
        
WM_feature_data_writer('ECG200',1,10, X_train_feature, y_train, X_test_feature, y_test, 1)    


for train_ratio in train_ratio_list:    
    # n_conf, windows, norm_mean = get_super_parameter(name)
    # X_train_ori, y_train_ori, X_test, y_test = TSC_data_loader(name)

    # Set the random seed
    random_seed = 42
    np.random.seed(random_seed)
    sss = StratifiedShuffleSplit(n_splits=10, test_size=1 - train_ratio, random_state=0)
    # sss.gc
    sss.get_n_splits(data, data_label)
    ind = 0
    for train_index, test_index in sss.split(X_train, y_train):
        print('X_train.shape:{}'.format(X_train.shape))
        print('train_index_shape: {}; test_index_shape:{}'.format(train_index.shape, test_index.shape))
        print('train_index: {}; test_index'.format(train_index, test_index))
        print('ind: {}'.format(ind))
        print('ratio: {}'.format(train_ratio))

        data = X_train[train_index,:]
        data_label = np.array(y_train)[train_index]

        skip = False
        for i in range(222):  # iterate all the features
          unique_elements, counts = np.unique(data[:, i, :], return_counts=True)
          # pdb.set_trace()
          if int(np.max(counts)) == data.shape[0] * 7:
            skip = True
            break
        
        if skip:
          ind = ind+1
          continue
        
        WM_model = WEASELMUSE(strategy='normal', drop_sum=True, window_sizes=[6], word_size=2, n_bins=2)
        # if ind == 8:
        #   pdb.set_trace()
        # pdb.set_trace()
        WM_model.fit(data, data_label)
        X_train_feature = WM_model.transform(data)
        X_test_feature = WM_model.transform(data_test)
        # y_predict = BOSSdistance_predict(X_test_feature, X_train_feature, y_train)
        # test_accuracy = sklearn.metrics.accuracy_score(np.squeeze(y_test), y_predict)
        
        
        # print(name, 'n_conf = ' ,n_conf, 'windows = ', windows, 'norm_mean = ',norm_mean)
        # print('pyts accuracy = ', test_accuracy, ' pyts_shape= = ',X_train_feature.shape)
        
        WM_feature_data_writer('ECG200', train_ratio,ind, X_train_feature, data_label, X_test_feature, data_test_label, 1)
        ind = ind+1

X_train.shape:(131, 222, 7)
train_index_shape: (13,); test_index_shape:(118,)
train_index: [120  77  25  61  46  85  52  62 114  95  66 123  75]; test_index
ind: 0
ratio: 0.1
X_train.shape:(131, 222, 7)
train_index_shape: (13,); test_index_shape:(118,)
train_index: [ 99  82   8 125 112  96  14 119  23  77  90  56   2]; test_index
ind: 1
ratio: 0.1
X_train.shape:(131, 222, 7)
train_index_shape: (13,); test_index_shape:(118,)
train_index: [ 57  20 116 112  96  58  70  82 101 130  90 127  83]; test_index
ind: 2
ratio: 0.1
X_train.shape:(131, 222, 7)
train_index_shape: (13,); test_index_shape:(118,)
train_index: [ 81  63 119  97 122 103  65 101  56  27  71  54  57]; test_index
ind: 3
ratio: 0.1
X_train.shape:(131, 222, 7)
train_index_shape: (13,); test_index_shape:(118,)
train_index: [ 94  19  18 106  10  33 117  25 104   9 112 123 122]; test_index
ind: 4
ratio: 0.1
X_train.shape:(131, 222, 7)
train_index_shape: (13,); test_index_shape:(118,)
train_index: [  6  91  72  36  28 111  35 112  